In [ ]:
# !pip install tensorflow==2.14.0
# !pip install update urllib3

# !pip install textattack
# !pip install evaluate

In [ ]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
import numpy as np
import evaluate
from datasets import load_dataset, DatasetDict
import torch
import transformers
import textattack
from textattack.models.helpers import T5ForTextToText
from textattack.models.tokenizers import T5Tokenizer
from textattack.models.wrappers.huggingface_model_wrapper import HuggingFaceModelWrapper
torch.cuda.empty_cache()

Functions

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

Load Dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT")

dataset = load_dataset("financial_phrasebank", 'sentences_50agree')
test = dataset['train'].train_test_split(test_size=0.3,seed=1997)
train = test['train'].train_test_split(test_size=0.2,seed=1997)
dataset = DatasetDict({
    'train': train['train'],
    'test': test['test'],
    'validation': train['test']})
dataset.save_to_disk("content/drive/MyDrive/NLP Project/data/")

training_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']
tokenized_training_data = training_data.map(tokenize_function, batched=True)
tokenized_validation_data = validation_data.map(tokenize_function, batched=True)
tokenized_test_data = test_data.map(tokenize_function, batched=True)


Saving the dataset (0/1 shards):   0%|          | 0/2713 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1454 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/679 [00:00<?, ? examples/s]

Train Model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT", num_labels=3)

training_args = TrainingArguments(output_dir = "test_trainer",evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_data,
    eval_dataset=tokenized_validation_data,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-no-defensive")

config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ahmedrachid/FinancialBERT and are newly initialized: ['bert.pooler.dense.bias', 'classifier.bias', 'bert.pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.397451,0.843888
2,0.412100,0.658348,0.824742
3,0.136200,0.831160,0.837997


Attack train data

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-no-defensive")
model_wrapper = HuggingFaceModelWrapper(model,tokenizer)
textattack_dataset = textattack.datasets.HuggingFaceDataset(dataset['train'])
attack = textattack.attack_recipes.PWWSRen2019.build(model_wrapper)
attack_args = textattack.AttackArgs(
    num_examples=-1,
    log_to_csv="content/drive/MyDrive/NLP Project/data/attacks_PWWSRen2019.csv",
)
attacker = textattack.Attacker(attack, textattack_dataset, attack_args)
attacks = attacker.attack_dataset()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path content/drive/MyDrive/NLP Project/data/attacks_PWWSRen2019.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/2713 [00:07<5:27:36,  7.25s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Neutral (100%)]] --> [[Negative (96%)]]

About 72 % of Evraz Group shares are [[owned]] by Lanebrook , whose beneficiaries , on the one hand , are Millhouse , the [[holding]] [[company]] for the assets of billionaire [[Roman]] Abramovich and his business partners 50 % ; and Evraz executives Alexander Abramov and Alexander Frolov 50 % , on the other .

About 72 % of Evraz Group shares are [[have]] by Lanebrook , whose beneficiaries , on the one hand , are Millhouse , the [[arrest]] [[companionship]] for the assets of billionaire [[papistical]] Abramovich and his business partners 50 % ; and Evraz executives Alexander Abramov and Alexander Frolov 50 % , on the other .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|          | 2/2713 [00:11<4:22:57,  5.82s/it]

...

[Succeeded / Failed / Skipped / Total] 297 / 99 / 3 / 399:  15%|█▍        | 399/2713 [27:18<2:38:24,  4.11s/it]

--------------------------------------------- Result 399 ---------------------------------------------
[[Positive (100%)]] --> [[Neutral (100%)]]

Cargotec Corporation , Press Release , August 26 , 2008 at 10 a.m. Finnish time Cargotec 's MacGREGOR business area providing marine cargo handling and offshore load handling solutions has [[received]] [[significant]] offshore crane retrofit order .

Cargotec Corporation , Press Release , August 26 , 2008 at 10 a.m. Finnish time Cargotec 's MacGREGOR business area providing marine cargo handling and offshore load handling solutions has [[standard]] [[pregnant]] offshore crane retrofit order .




[Succeeded / Failed / Skipped / Total] 298 / 99 / 3 / 400:  15%|█▍        | 400/2713 [27:22<2:38:17,  4.11s/it]

--------------------------------------------- Result 400 ---------------------------------------------
[[Positive (100%)]] --> [[Neutral (100%)]]

After the transaction , Alma Media [[raised]] its stake in Talentum to 30.65 % of the shares and some 31.12 % of voting rights .

After the transaction , Alma Media [[parent]] its stake in Talentum to 30.65 % of the shares and some 31.12 % of voting rights .




KeyboardInterrupt: ignored

Load perturbed data

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/NLP Project/data/attacks_PWWSRen2019.csv")
df = pd.DataFrame(df)

df = pd.read_csv("/content/drive/MyDrive/NLP Project/data/attacks_PWWSRen2019.csv")
df = pd.DataFrame(df)
orgininal_training_data = df[["original_text","ground_truth_output"]]
orgininal_training_data.rename(columns={"original_text":"sentence","ground_truth_output":"label"},inplace=True)
perturbed_training_data = df[df["result_type"]=="Successful"][["perturbed_text","ground_truth_output"]]
perturbed_training_data.rename(columns={"perturbed_text":"sentence","ground_truth_output":"label"}, inplace=True)
perturbed_training_data = pd.concat([orgininal_training_data,perturbed_training_data])

perturbed_training_data['sentence'] = perturbed_training_data['sentence'].str.replace("\[\[","")
perturbed_training_data['sentence'] = perturbed_training_data['sentence'].str.replace("\]\]","")

perturbed_training_data = Dataset.from_pandas(perturbed_training_data)
tokenized_perturbed_training_data = perturbed_training_data.map(tokenize_function, batched=True)

Defend Model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT")

training_args = TrainingArguments(output_dir = "test_trainer",evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_perturbed_training_data,
    eval_dataset=tokenized_validation_data,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-defended-PWWSRen2019")


Results

In [ ]:
textattack_dataset = textattack.datasets.HuggingFaceDataset(dataset['test'])

model = AutoModelForSequenceClassification.from_pretrained("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-no-defensive")
model_wrapper = HuggingFaceModelWrapper(model,tokenizer)
attack = textattack.attack_recipes.PWWSRen2019.build(model_wrapper)
attack_args = textattack.AttackArgs(
    num_examples=-1
)
attacker = textattack.Attacker(attack, textattack_dataset, attack_args)
attacks = attacker.attack_dataset()

In [ ]:
textattack_dataset = textattack.datasets.HuggingFaceDataset(dataset['test'])

model = AutoModelForSequenceClassification.from_pretrained("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-defended-PWWSRen2019")
model_wrapper = HuggingFaceModelWrapper(model,tokenizer)
attack = textattack.attack_recipes.PWWSRen2019.build(model_wrapper)
attack_args = textattack.AttackArgs(
    num_examples=-1
)
attacker = textattack.Attacker(attack, textattack_dataset, attack_args)
attacks = attacker.attack_dataset()